In [3]:
import pandas as pd

# Carregue o DataFrame students_performance.csv
caminho_arquivo = '../data/datasets/students_performance.csv' 
df = pd.read_csv(caminho_arquivo)

# Crie uma cópia para o pré-processamento. Trabalhe sempre na cópia (df_prep)
df_prep = df.copy()

# Verifique os valores ausentes
print("Valores Ausentes Antes da Imputação:")
print(df_prep.isnull().sum())

Valores Ausentes Antes da Imputação:
student_id              0
age                     0
gender                  0
parental_education      0
study_hours_week      293
attendance_rate       232
extracurricular         0
sleep_hours           266
previous_scores       127
tutoring                0
internet_quality      155
family_income         278
health_status           0
final_grade             0
dtype: int64


In [4]:
# 1. Imputação pela Média para 'study_hours_week'
media_horas_estudo = df_prep['study_hours_week'].mean()
df_prep['study_hours_week'].fillna(media_horas_estudo, inplace=True)

# 2. Imputação pela Média para 'sleep_hours'
media_horas_sono = df_prep['sleep_hours'].mean()
df_prep['sleep_hours'].fillna(media_horas_sono, inplace=True)

# 3. Verificação: O resultado deve mostrar 0 para essas colunas
print("\nValores Ausentes Após Imputação (Horas de Estudo e Sono):")
print(df_prep[['study_hours_week', 'sleep_hours']].isnull().sum())


Valores Ausentes Após Imputação (Horas de Estudo e Sono):
study_hours_week    0
sleep_hours         0
dtype: int64


In [5]:
# Lista de colunas categóricas com Missing Values:
colunas_categoricas_com_nan = [
    'extracurricular', 
    'internet_quality', 
    'family_income', 
    'health_status'
]

for coluna in colunas_categoricas_com_nan:
    # Calcula a moda (valor mais frequente)
    moda = df_prep[coluna].mode()[0]
    
    # Preenche os valores ausentes (NaN) com a moda
    df_prep[coluna].fillna(moda, inplace=True)

# Verificação: o resultado deve mostrar 0 para todas as colunas
print("Valores Ausentes após Imputação (Colunas Categóricas):")
print(df_prep[colunas_categoricas_com_nan].isnull().sum())

Valores Ausentes após Imputação (Colunas Categóricas):
extracurricular     0
internet_quality    0
family_income       0
health_status       0
dtype: int64


In [6]:
# 1. Identifica todas as colunas do tipo 'object' (texto)
colunas_categoricas = df_prep.select_dtypes(include='object').columns.tolist()

print("Colunas Categóricas a serem transformadas:", colunas_categoricas)

# 2. Aplica One-Hot Encoding (pd.get_dummies)
# Ele transforma cada categoria em uma nova coluna de 0s e 1s
df_prep = pd.get_dummies(df_prep, columns=colunas_categoricas, drop_first=True)

# 3. Verificação: O DataFrame deve ter mais colunas, e todas devem ser numéricas
print("\nPrimeiras 5 linhas do DataFrame após a transformação:")
print(df_prep.head())
print("\nNova contagem de colunas:", df_prep.shape[1])

Colunas Categóricas a serem transformadas: ['student_id', 'gender', 'parental_education', 'extracurricular', 'tutoring', 'internet_quality', 'family_income', 'health_status']

Primeiras 5 linhas do DataFrame após a transformação:
   age  study_hours_week  attendance_rate  sleep_hours  previous_scores  \
0   22              5.66            69.55         6.49            49.41   
1   19             13.30            58.82         8.12            50.29   
2   25             10.43            59.72         6.60            71.64   
3   21              3.90            68.33         6.99            50.93   
4   19              4.24            50.44         8.06            54.00   

   final_grade  student_id_STD00002  student_id_STD00003  student_id_STD00004  \
0        85.39                False                False                False   
1        98.43                False                False                False   
2       100.00                False                False                Fals

In [8]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 1. REMOVER TODAS AS COLUNAS DE ID QUE FORAM CRIADAS PELO pd.get_dummies
# Usa o filtro 'filter' para selecionar todas as colunas que começam com 'student_id' e as remove.
colunas_id = [col for col in df_prep.columns if col.startswith('student_id')]
df_prep = df_prep.drop(columns=colunas_id)

# 2. SEPARAR O TARGET (Y) DAS FEATURES (X)
target_column = 'final_grade'

X = df_prep.drop(columns=[target_column])
y = df_prep[target_column]

# 3. ESCALONAMENTO/NORMALIZAÇÃO (Garante que os números estejam na mesma escala 0-1)
scaler = MinMaxScaler()
# Aplica o escalonamento apenas às features (X)
X_scaled = scaler.fit_transform(X)

# Transforma X_scaled de volta para um DataFrame para fácil visualização
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 4. Verificação final da limpeza
print("--- DataFrame Final Limpo e Escalonado (Primeiras Linhas) ---")
print(X_scaled_df.head())
print(f"\nDimensões Finais (X, y): {X_scaled_df.shape} e {y.shape}")

--- DataFrame Final Limpo e Escalonado (Primeiras Linhas) ---
        age  study_hours_week  attendance_rate  sleep_hours  previous_scores  \
0  0.060976          0.079438         0.552933     0.318444         0.434669   
1  0.024390          0.144838         0.431181     0.435879         0.448228   
2  0.097561          0.120271         0.441393     0.326369         0.777196   
3  0.048780          0.064373         0.539090     0.354467         0.458089   
4  0.024390          0.067283         0.336094     0.431556         0.505393   

   gender_  F   gender_  M   gender_F  gender_Female  gender_M  ...  \
0          0.0          0.0       0.0            0.0       1.0  ...   
1          0.0          0.0       0.0            0.0       1.0  ...   
2          0.0          0.0       0.0            0.0       1.0  ...   
3          0.0          0.0       1.0            0.0       0.0  ...   
4          0.0          0.0       1.0            0.0       0.0  ...   

   family_income_MEDIUM  famil

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd # Importa novamente para garantir que está disponível, embora já devesse estar

# 1. REMOVER TODAS AS COLUNAS DE ID QUE FORAM CRIADAS PELO pd.get_dummies
# Filtra todas as colunas que começam com 'student_id' e as remove.
colunas_id = [col for col in df_prep.columns if col.startswith('student_id')]
# Usamos 'axis=1' para garantir que estamos removendo colunas
df_prep = df_prep.drop(columns=colunas_id)

# 2. SEPARAR O TARGET (Y) DAS FEATURES (X)
target_column = 'final_grade'

X = df_prep.drop(columns=[target_column], axis=1) # axis=1 garante que remove colunas
y = df_prep[target_column]

# 3. ESCALONAMENTO/NORMALIZAÇÃO (Garante que os números estejam na mesma escala 0-1)
scaler = MinMaxScaler()
# Aplica o escalonamento apenas às features (X)
X_scaled = scaler.fit_transform(X)

# Transforma X_scaled de volta para um DataFrame para fácil visualização
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# 4. Verificação final da limpeza
print("--- DataFrame Final Limpo e Escalonado (Primeiras Linhas) ---")
print(X_scaled_df.head())
print(f"\nDimensões Finais (X, y): {X_scaled_df.shape} e {y.shape}")

--- DataFrame Final Limpo e Escalonado (Primeiras Linhas) ---
        age  study_hours_week  attendance_rate  sleep_hours  previous_scores  \
0  0.060976          0.079438         0.552933     0.318444         0.434669   
1  0.024390          0.144838         0.431181     0.435879         0.448228   
2  0.097561          0.120271         0.441393     0.326369         0.777196   
3  0.048780          0.064373         0.539090     0.354467         0.458089   
4  0.024390          0.067283         0.336094     0.431556         0.505393   

   gender_  F   gender_  M   gender_F  gender_Female  gender_M  ...  \
0          0.0          0.0       0.0            0.0       1.0  ...   
1          0.0          0.0       0.0            0.0       1.0  ...   
2          0.0          0.0       0.0            0.0       1.0  ...   
3          0.0          0.0       1.0            0.0       0.0  ...   
4          0.0          0.0       1.0            0.0       0.0  ...   

   family_income_MEDIUM  famil